In [1]:
import pandas as pd
# ensure that all columns are shown and that colum content is not cut
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width',1000)
pd.set_option('display.max_rows', 500) # ensure that all rows are shown

# Collector Deep Dive
This notebooks dives a little deeper into using the collector classes.

## Basics

All the `Collector` classes have their own factory method(s) which instantiates the class. Most of these factory methods
also provide parameters to filter the data directly when being loaded from the parquet files.
These are
* the `forms_filter` <br> lets you select which report type should be loaded (e.g. "10-K" or "10-Q").<br>
  Note: the fomrs filter affects all dataframes (sub, pre, num).
* the `stmt_filter` <br> defines the statements that should be loaded (e.g., "BS" if only "Balance Sheet" data should be loaded) <br>
  Note: the stmt filter only affects the pre dataframe.
* the `tag_filter` <br> defines the tags, that should be loaded (e.g., "Assets" if only the "Assets" tag should be loaded) <br>
  Note: the tag filter affects the pre and num dataframes.

It is also possible to apply filter for these attributes after the data is loaded, but since the `Collector` classes
apply this filters directly during the load process from the parquet files (which means that fewer data is loaded from
the disk and also the memory footprint is reduced) this is generally more efficient.

All `Collector` classes have a `collect` method which then loads the data from the parquet files and returns an instance
of `RawDataBag`. The `RawDataBag` instance contains then a pandas dataframe for the `sub` (subscription) data,
`pre` (presentation) data, and `num` (the numeric values) data.

## `SingleReportCollector`
As the name suggests, this `Collector` returns the data of a single report. It is instantiated by providing the `adsh` of the desired report as parameter of the `get_report_by_adsh` factory method, 
or by using an instance of the `IndexReport` as parameter of the `get_report_by_indexreport`. (As a reminder: instances of `IndexReport` are returned by the `CompanyIndexReader` class).

Reading a single report: **Apples 10-K from 2022**

In [2]:
from secfsdstools.e_collector.reportcollecting import SingleReportCollector

apple_10k_2022_adsh = "0000320193-22-000108"

collector: SingleReportCollector = SingleReportCollector.get_report_by_adsh(adsh=apple_10k_2022_adsh)
rawdatabag = collector.collect()

# as expected, there is just one entry in the submission dataframe
print("sub", rawdatabag.sub_df.shape)

# just print the size of the pre and num dataframes
print("pre", rawdatabag.pre_df.shape)
print("num", rawdatabag.num_df.shape)

# joining the pre and num dataframes
joineddatabag = rawdatabag.join()
print("pre_num", joineddatabag.pre_num_df.shape)

2025-02-01 07:34:13,223 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-01 07:34:13,278 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg


sub (1, 36)
pre (100, 10)
num (437, 10)
pre_num (509, 17)


As mentioned above, we can also directly apply filters, to reduce the amount of data that is loaded. 

First, let's only load data for the **Balance Sheet** by using the `stmt_filter`.

In [3]:
from secfsdstools.e_collector.reportcollecting import SingleReportCollector

collector: SingleReportCollector = SingleReportCollector.get_report_by_adsh(adsh=apple_10k_2022_adsh, stmt_filter=['BS'])
rawdatabag = collector.collect()

# as expected, there is just one entry in the submission dataframe
print("sub", rawdatabag.sub_df.shape)

# just print the size of the pre and num dataframes
print("pre", rawdatabag.pre_df.shape)
print("num", rawdatabag.num_df.shape)

# joining the pre and num dataframes
joineddatabag = rawdatabag.join()
print("pre_num", joineddatabag.pre_num_df.shape)

2025-02-01 07:34:57,913 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg


sub (1, 36)
pre (32, 10)
num (437, 10)
pre_num (171, 17)


As mentioned above, the stmt_filter only applies to the pre_df, since only the pre_df has information about the statement which a tag belongs to. But of course, also the joined dataframe is significantely smaller.

Next, lets even be a bit more restrictive and just load the **'Assets'** tag by using the `tag_filter`.

In [4]:
from secfsdstools.e_collector.reportcollecting import SingleReportCollector

collector: SingleReportCollector = SingleReportCollector.get_report_by_adsh(adsh=apple_10k_2022_adsh, tag_filter=['Assets'])
rawdatabag = collector.collect()

# as expected, there is just one entry in the submission dataframe
print("sub", rawdatabag.sub_df.shape)

# just print the size of the pre and num dataframes
print("pre", rawdatabag.pre_df.shape)
print("num", rawdatabag.num_df.shape)

# joining the pre and num dataframes
joineddatabag = rawdatabag.join()
print("pre_num", joineddatabag.pre_num_df.shape)

2025-02-01 07:35:23,207 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg


sub (1, 36)
pre (1, 10)
num (2, 10)
pre_num (2, 17)


Now, as expected, there should only be one Asset tag in the pre dataframe. The reason with have two entries in the num dataframe is, that there is a value for the year 2022 and a value for the previous year 2021, as can be seen in ddate column:

In [5]:
rawdatabag.num_df

,adsh,tag,version,ddate,qtrs,uom,segments,coreg,value,footnote
0,0000320193-22-000108,Assets,us-gaap/2022,20220930,0,USD,,,3.527550e+11,None
1,0000320193-22-000108,Assets,us-gaap/2022,20210930,0,USD,,,3.510020e+11,None


# `MultiReportCollector`
Contrary to the `SingleReportCollector`, this `Collector` can collect data from several
reports. Moreover, the data of the reports are loaded in parallel, this  especially improves the performance if the
reports are from different quarters (resp. are in different zip files). The class provides the factory methods 
`get_reports_by_adshs` and `get_reports_by_indexreports`. The first takes a list of adsh strings, the second a list
of `IndexReport` instances.

Reading two reports: **Apple's 10-K from 2022 and 2012**

In [6]:
from secfsdstools.e_collector.multireportcollecting import MultiReportCollector
apple_10k_2022_adsh = "0000320193-22-000108"
apple_10k_2012_adsh = "0001193125-12-444068"

# load only the assets tags that are present in the 10-K report of apple in the years
# 2022 and 2012
collector: MultiReportCollector = \
    MultiReportCollector.get_reports_by_adshs(adshs=[apple_10k_2022_adsh,
                                                     apple_10k_2012_adsh])
rawdatabag = collector.collect()
# as expected, there are just two entries in the submission dataframe
print("sub", rawdatabag.sub_df.shape)

# just print the size of the pre and num dataframes
print("pre", rawdatabag.pre_df.shape)
print("num", rawdatabag.num_df.shape)

# joining the pre and num dataframes
joineddatabag = rawdatabag.join()
print("pre_num", joineddatabag.pre_num_df.shape)

2025-02-01 07:35:46,930 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-01 07:35:46,934 [INFO] parallelexecution      items to process: 2
2025-02-01 07:35:49,200 [INFO] parallelexecution      commited chunk: 0


sub (2, 36)
pre (191, 10)
num (955, 10)
pre_num (1169, 17)


Again, using the filter parameters reduces the amount of data that is loaded. Let us load the tags **'Assets' and 'Liabilities'.**

In [7]:
from secfsdstools.e_collector.multireportcollecting import MultiReportCollector
apple_10k_2022_adsh = "0000320193-22-000108"
apple_10k_2012_adsh = "0001193125-12-444068"

# load only the assets tags that are present in the 10-K report of apple in the years
# 2022 and 2012
collector: MultiReportCollector = \
    MultiReportCollector.get_reports_by_adshs(adshs=[apple_10k_2022_adsh, apple_10k_2012_adsh],
                                              tag_filter=['Assets', 'Liabilities'])
rawdatabag = collector.collect()
# as expected, there are just two entries in the submission dataframe
print("sub", rawdatabag.sub_df.shape)

# just print the size of the pre and num dataframes
print("pre", rawdatabag.pre_df.shape)
print("num", rawdatabag.num_df.shape)

# joining the pre and num dataframes
joineddatabag = rawdatabag.join()
print("pre_num", joineddatabag.pre_num_df.shape)

2025-02-01 07:35:57,479 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-01 07:35:57,483 [INFO] parallelexecution      items to process: 2
2025-02-01 07:35:59,620 [INFO] parallelexecution      commited chunk: 0


sub (2, 36)
pre (4, 10)
num (22, 10)
pre_num (22, 17)


In [8]:
rawdatabag.num_df

,adsh,tag,version,ddate,qtrs,uom,segments,coreg,value,footnote
0,0000320193-22-000108,Assets,us-gaap/2022,20220930,0,USD,,,3.527550e+11,None
1,0000320193-22-000108,Liabilities,us-gaap/2022,20220930,0,USD,,,3.020830e+11,None
2,0000320193-22-000108,Assets,us-gaap/2022,20210930,0,USD,,,3.510020e+11,None
3,0000320193-22-000108,Liabilities,us-gaap/2022,20210930,0,USD,,,2.879120e+11,None
4,0001193125-12-444068,Assets,us-gaap/2012,20110930,0,USD,BusinessSegments=AsiaAndPacific;,,1.710000e+09,None
5,0001193125-12-444068,Assets,us-gaap/2012,20120930,0,USD,,,1.760640e+11,None
6,0001193125-12-444068,Assets,us-gaap/2012,20110930,0,USD,BusinessSegments=Retail;,,2.151000e+09,None
7,0001193125-12-444068,Assets,us-gaap/2012,20110930,0,USD,BusinessSegments=Europe;,,1.520000e+09,None
8,0001193125-12-444068,Assets,us-gaap/2012,20120930,0,USD,BusinessSegments=UnallocatedAmountToSegment;,,1.607870e+11,None
9,0001193125-12-444068,Assets,us-gaap/2012,20110930,0,USD,,,1.163710e+11,None


As expected, the data now contains the values for 'Assets' and 'Liabilities' for the year 2012 and the previous year 2011, as well as for the year 2022 and the previoius year 2021. Note that we also have additional `segments` information.

# `CompanyReportCollector`

This class returns reports for one or more companies. The factory method `get_company_collector` provides the parameter `ciks` which takes a list of cik numbers.

Let us read the data for **all reports of Apple and Microsoft.** 

**Note**: To do that, the framework has to read data from all quarters, so this may take a few seconds.

In [9]:
from secfsdstools.e_collector.companycollecting import CompanyReportCollector

apple_cik = 320193
microsoft_cik = 789019
collector = CompanyReportCollector.get_company_collector(ciks=[apple_cik, microsoft_cik])

rawdatabag = collector.collect()

print("sub", rawdatabag.sub_df.shape)

# just print the size of the pre and num dataframes
print("pre", rawdatabag.pre_df.shape)
print("num", rawdatabag.num_df.shape)

# joining the pre and num dataframes
joineddatabag = rawdatabag.join()
print("pre_num", joineddatabag.pre_num_df.shape)

2025-02-01 07:37:13,169 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-01 07:37:13,380 [INFO] parallelexecution      items to process: 62
2025-02-01 07:37:33,484 [INFO] parallelexecution      commited chunk: 0


sub (129, 36)
pre (12938, 10)
num (58105, 10)
pre_num (71713, 17)


As you will see, this takes a couple dozens of seconds. But nonetheless, data from all zip files was loaded in parallel.

But maybe, we just want to have a look at the **'Assets' of all 10-K reports.**

In [10]:
from secfsdstools.e_collector.companycollecting import CompanyReportCollector

collector = CompanyReportCollector.get_company_collector(ciks=[apple_cik, microsoft_cik],
                                                        tag_filter=['Assets'],
                                                        forms_filter=['10-K'])

rawdatabag = collector.collect()

print("sub", rawdatabag.sub_df.shape)

# just print the size of the pre and num dataframes
print("pre", rawdatabag.pre_df.shape)
print("num", rawdatabag.num_df.shape)

# joining the pre and num dataframes
joineddatabag = rawdatabag.join()
print("pre_num", joineddatabag.pre_num_df.shape)

2025-02-01 07:39:26,087 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-01 07:39:26,309 [INFO] parallelexecution      items to process: 31
2025-02-01 07:39:36,260 [INFO] parallelexecution      commited chunk: 0


sub (31, 36)
pre (31, 10)
num (131, 10)
pre_num (131, 17)


# `ZipCollector`

This `Collector` collects the data of one or more zip (resp. the folders that contain the parquet
  files of this zip files). And since every of the original zip files contains the data for one quarter, the names you provide
  in the `get_zip_by_name` or `get_zip_by_names` factory methods reflect the quarter which data you want to load: e.g. `2022q1.zip`.
  
There are several fatctory methods to provide the functionality. First let us load the data for the zip file **2022q1.zip**.

In [11]:
from secfsdstools.e_collector.zipcollecting import ZipCollector

collector: ZipCollector = ZipCollector.get_zip_by_name(name="2022q1.zip")

rawdatabag = collector.collect()

print("sub", rawdatabag.sub_df.shape)

# just print the size of the pre and num dataframes
print("pre", rawdatabag.pre_df.shape)
print("num", rawdatabag.num_df.shape)

# joining the pre and num dataframes
joineddatabag = rawdatabag.join()
print("pre_num", joineddatabag.pre_num_df.shape)

2025-02-01 07:39:50,210 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-01 07:39:50,215 [INFO] parallelexecution      items to process: 1
2025-02-01 07:39:50,216 [INFO] zipcollecting  processing C:\Users\hansj\secfsdstools\data\parquet\quarter\2022q1.zip
2025-02-01 07:39:52,483 [INFO] parallelexecution      commited chunk: 0


sub (7237, 36)
pre (832867, 10)
num (3264632, 10)
pre_num (4208329, 17)


As you may notice this is quite a significant amount of data that was loaded, just for one single quarter. 

Next, we are going to load data for **all the quarters of 2022, but only the Balance Sheet of the 10-K reports.**

In [12]:
from secfsdstools.e_collector.zipcollecting import ZipCollector

collector: ZipCollector = ZipCollector.get_zip_by_names(names=["2022q1.zip", "2022q2.zip", "2022q3.zip", "2022q4.zip"],
                                                        forms_filter=["10-K"],
                                                        stmt_filter=["BS"],)

rawdatabag = collector.collect()

print("sub", rawdatabag.sub_df.shape)

# just print the size of the pre and num dataframes
print("pre", rawdatabag.pre_df.shape)
print("num", rawdatabag.num_df.shape)

# joining the pre and num dataframes
joineddatabag = rawdatabag.join()
print("pre_num", joineddatabag.pre_num_df.shape)

2025-02-01 07:40:21,535 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-01 07:40:21,539 [INFO] parallelexecution      items to process: 4
2025-02-01 07:41:02,066 [INFO] parallelexecution      commited chunk: 0


sub (6516, 36)
pre (241088, 10)
num (2818850, 10)
pre_num (947786, 17)


But we can even be a little more bold and read data from **all zip files at once**. We will read **10-K and 10-Q reports, but read only the Assets tag.** This will take some time.

Note: Use with caution, since this can fill up your memory if you don't provide a tag_filter.

In [13]:
from secfsdstools.e_collector.zipcollecting import ZipCollector

collector: ZipCollector = ZipCollector.get_all_zips(forms_filter=["10-K", "10-Q"],
                                                    tag_filter=["Assets"])

rawdatabag = collector.collect()

print("sub", rawdatabag.sub_df.shape)

# just print the size of the pre and num dataframes
print("pre", rawdatabag.pre_df.shape)
print("num", rawdatabag.num_df.shape)

# joining the pre and num dataframes
joineddatabag = rawdatabag.join()
print("pre_num", joineddatabag.pre_num_df.shape)

2025-02-01 07:41:04,032 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-01 07:41:04,046 [INFO] parallelexecution      items to process: 63
2025-02-01 07:41:41,665 [INFO] parallelexecution      commited chunk: 0


sub (350043, 36)
pre (350347, 10)
num (1595985, 10)
pre_num (1654978, 17)


## `post_load_filter`
The ZipCollector factory methods have an additional filter parameter: `post_load_filter`.

Loading the from multiple zip files is done in parallel processes. The number of parallel prcocesses depends on the number of core the system has. First, a RawDataBag instance is created for every zip file that has to be loaded and once all the zip files are loaded, all these RawDataBag instances are concatenated into a single instance of a RawDataBag. After that, additional filters can be applied with the `filter` method of the RawDataBag.

Especially when wanting to load data from all zip files at once, it would make sense to have a possibility to add an additional filter directly after the data of a single zip file has been loaded, since this would reduce the memory footprint significantly.

This is what the `post_laod_filter` is for. It simply takes a function as parameter that receives a `RawDataBag`as parameter and returns a `RawDataBag` as result. Hence, it can also be defined as a lambda function.

The following example will load the data for all available 10-K and 10-Q balance sheets. If we would do that
without a `post_laod_filter` it might very likely in a out-of-memory exception before we change to reduce the data. Therfore, we add a `post_load_filter` that

* filters only the datapoints of the current report (by using `ReportPeriodRawFilter`)
* removes datapointss ob subsidiaries (by using `MainCoregFilter`)
* create a real copy of the reduced dataframes so that dataframes containing all the data can be garbage collected (by using the `copy_bag()`) method

The `post_load_filter` is just a function that receives a `RawDataBag` and has to return a `RawDataBag`. It can be either defined as a function or
directly as a lambda expression.

```
    # as function
    def postloadfilter(databag: RawDataBag) -> RawDataBag:
        return databag[ReportPeriodRawFilter()][MainCoregFilter()]
    
    # as lambda
    post_filter = lambda x: x[ReportPeriodRawFilter()][MainCoregFilter()]
```

**Attention 1:** while either defining a function or using a lambda did work perfectly when running the script directly from the command line or in an IDE, it didn't work within Jupyter. In Jupyter, I have to use a function and moreover, also the needed imports have to be included in the function itself:
```
def postloadfilter(databag: RawDataBag) -> RawDataBag:
    from secfsdstools.e_filter.rawfiltering import ReportPeriodRawFilter, MainCoregFilter
    return databag[ReportPeriodRawFilter()][MainCoregFilter()]
```

**Attention 2:** running this code took a few minutes on my 4-core/32GB laptop (around 6 minutes). Moreover, it still needed about 18GB (!) of free memory when launched. Open the TaskManager and watch your CPUs and Memory working.

In [14]:
import os

from secfsdstools.e_collector.zipcollecting import ZipCollector
from secfsdstools.d_container.databagmodel import RawDataBag
from secfsdstools.e_filter.rawfiltering import ReportPeriodRawFilter, MainCoregRawFilter

target_path = "bs_10k_10q_all_joined"
os.makedirs(target_path, exist_ok = True)

def postloadfilter(databag: RawDataBag) -> RawDataBag:
    from secfsdstools.e_filter.rawfiltering import ReportPeriodRawFilter, MainCoregRawFilter
    return databag[ReportPeriodRawFilter()][MainCoregRawFilter()]

collector: ZipCollector = ZipCollector.get_all_zips(forms_filter=["10-K", "10-Q"],
                                                    stmt_filter=["BS"],
                                                    post_load_filter=postloadfilter)

rawdatabag = collector.collect()

print("sub", rawdatabag.sub_df.shape)

# just print the size of the pre and num dataframes
print("pre", rawdatabag.pre_df.shape)
print("num", rawdatabag.num_df.shape)

print("----------------------------------------")

# joining the pre and num dataframes
joineddatabag = rawdatabag.join()
print("pre_num", joineddatabag.pre_num_df.shape)

print("----------------------------------------")

# of course, saving the the databag would be a good idea here
# but remember, the path has to exist and has to be empty
joineddatabag.save(target_path=target_path)

2025-02-01 07:46:15,435 [INFO] configmgt  reading configuration from C:\Users\hansj\.secfsdstools.cfg
2025-02-01 07:46:15,439 [INFO] parallelexecution      items to process: 63
2025-02-01 07:52:05,827 [INFO] parallelexecution      commited chunk: 0


sub (350043, 36)
pre (12431739, 10)
num (59042640, 10)
----------------------------------------
pre_num (20508786, 17)
----------------------------------------


sub (316161, 36)
pre (14085879, 10)
num (51855742, 9)
pre_num (10723938, 16)